In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel("C:/Users/usman/OneDrive/Desktop/dubts/emails.xlsx")

In [ ]:
web_links=df['Website Link']

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import openpyxl

def scrape_emails(website_links):
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    email_data = []

    timeout = 15  # Connection timeout in seconds

    for link in website_links:
        try:
            response = requests.get(link, timeout=timeout)
            soup = BeautifulSoup(response.content, 'html.parser')

            main_email = re.search(email_regex, soup.text)
            if main_email:
                email_data.append((link, main_email.group()))
                print(f"Main email address found on {link}: {main_email.group()}")
            else:
                email_data.append((link, 'Email not found'))
                print(f"No email address found on {link}")
        except requests.exceptions.RequestException as e:
            email_data.append((link, 'Error retrieving website'))
            print(f"Error retrieving {link}: {e}")
        except requests.exceptions.Timeout:
            email_data.append((link, 'Connection timeout'))
            print(f"Connection timeout for {link}")

    return email_data

# Example usage:

email_data = scrape_emails(web_links)

# Create an Excel workbook and sheet
workbook = openpyxl.Workbook()
sheet = workbook.active

# Write headers
sheet['A1'] = 'Website Link'
sheet['B1'] = 'Email Address'

# Write email data to the sheet
for row, data in enumerate(email_data, start=2):
    website_link, email_address = data
    sheet.cell(row=row, column=1, value=website_link)
    sheet.cell(row=row, column=2, value=email_address)

# Save the workbook
workbook.save('email_data.xlsx')
